In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import t
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import torch

import logging
logging.disable(logging.WARNING)
import warnings
warnings.filterwarnings("ignore")

In [2]:
%local-changes

In [3]:
from mint.transfer_learning.engression_python import engression
from mint.transfer_learning.quantile_match import quantile_matching_estimate
from mint.transfer_learning.covariate_shift import kernel_mean_matching

### Concept and Covariate Shifts (Multi-source)
$P^{(0)}(x) \neq P^{(k)}(x)$ and $P^{(0)}(y|x) \neq P^{(k)}(y|x)$.

Source domain: $Y^{(1)}= \sin(3\beta^TX) + 1 + \epsilon$; 
and $Y^{(2)}= \cos(3\beta^TX) + 1 + \epsilon$ with $X\sim N(\mathbf{1}_d, I_d)$.

Target domain: $Y= \sin(3\beta^TX)/3 - 3 + \epsilon$ with $X\sim N(\mathbf{0}_d, 0.5^2 I_d)$.

In [4]:
# d = 5
# n_s = 1000
# # Source data
# np.random.rand(123)
# Sigma = np.eye(d)
# mu1 = np.ones(d)
# sig = 0.5
# X_dat1 = np.random.multivariate_normal(mean=mu1, cov=Sigma, size=n_s)
# beta1 = 1 / np.arange(1, d+1)
# Y1 = np.sin(3*np.dot(X_dat1, beta1)) + 1 + np.random.randn(n_s)*sig

# X_dat2 = np.random.multivariate_normal(mean=mu1, cov=Sigma, size=n_s)
# Y2 = 2*np.cos(3*np.dot(X_dat2, beta1)) + 1 + np.random.randn(n_s)*sig

# # Target data
# n0 = 50
# mu0 = np.zeros(d)
# X_dat0 = np.random.multivariate_normal(mean=mu0, cov=0.25*Sigma, size=n0)
# Y0 = np.sin(3*np.dot(X_dat0, beta1))/3 - 1 + np.random.randn(n0)*sig

# X_dat0_full = np.random.multivariate_normal(mean=mu0, cov=0.25*Sigma, size=2*n_s+n0)
# Y0_full = np.sin(3*np.dot(X_dat0_full, beta1))/3 - 1 + np.random.randn(2*n_s+n0)*sig

# X_test0 = np.random.multivariate_normal(mean=mu0, cov=0.25*Sigma, size=5000)
# Y0_test = np.sin(3*np.dot(X_test0, beta1))/3 - 1 + np.random.randn(5000)*sig

def sim_data(n_s=1000, n_0=50, n_test=5000, d=5, sig=0.5, mu_s=np.ones(5), mu_t=np.zeros(5), Sigma=np.eye(5), beta1=1/np.arange(1, 6)):
    # Target data
    X_dat0 = np.random.multivariate_normal(mean=mu_t, cov=0.25*Sigma, size=n_0)
    Y0 = np.sin(3*np.dot(X_dat0, beta1))/3 - 1 + np.random.randn(n_0)*sig
    dat0 = np.column_stack([Y0, X_dat0])
    
    # Source data
    X_dat1 = np.random.multivariate_normal(mean=mu_s, cov=Sigma, size=n_s)
    Y1 = np.sin(3*np.dot(X_dat1, beta1)) + 1 + np.random.randn(n_s)*sig
    dat1 = np.column_stack([Y1, X_dat1])

    X_dat2 = np.random.multivariate_normal(mean=mu_s, cov=Sigma, size=n_s)
    Y2 = 2*np.cos(3*np.dot(X_dat2, beta1)) + 1 + np.random.randn(n_s)*sig
    dat2 = np.column_stack([Y2, X_dat2])

    dat_source = [dat1, dat2]

    X_dat0_full = np.random.multivariate_normal(mean=mu_t, cov=0.25*Sigma, size=2*n_s+n_0)
    Y0_full = np.sin(3*np.dot(X_dat0_full, beta1))/3 - 1 + np.random.randn(2*n_s+n_0)*sig
    dat0_full = np.column_stack([Y0_full, X_dat0_full])

    X_test0 = np.random.multivariate_normal(mean=mu_t, cov=0.25*Sigma, size=n_test)
    Y0_test = np.sin(3*np.dot(X_test0, beta1))/3 - 1 + np.random.randn(n_test)*sig
    dat_test0 = np.column_stack([Y0_test, X_test0])

    return dat_source, dat0, dat0_full, dat_test0

In [5]:
import os
os.chdir("/data/sandcastle/boxes/fbsource/fbcode/mint/transfer_learning")

## Target-only ML regression

In [72]:
import os
os.chdir("/data/sandcastle/boxes/fbsource/fbcode/mint/transfer_learning")

B = 100
for n_s in [500, 1000, 2000, 5000]:
    res_full = pd.DataFrame()
    xbg_mse = np.zeros(B)
    krr_mse = np.zeros(B)
    nn_mse = np.zeros(B)
    for b in range(B):
        d = 5
        np.random.seed(b)
        dat_source, dat0, dat0_full, dat_test = sim_data(n_s=n_s, n_0=50, n_test=2000, d=d, sig=0.5, mu_s=np.ones(d), mu_t=np.zeros(d), Sigma=np.eye(d), beta1=1/np.arange(1, d+1))

        # Target-only ML models
        X0 = dat0[:, 1:]
        Y0 = dat0[:, 0]
        X_test = dat_test[:, 1:]
        Y_test = dat_test[:, 0]

        ## XGBoost
        param_grid = {
            'learning_rate': [0.001, 0.01, 0.1],
            'n_estimators': [10, 50, 100], 
            'max_depth': [3, 5],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
        }
        xgb_model = XGBRegressor(objective='reg:squarederror', random_state=0)
        grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X0, Y0)
        target_only_xgb = grid_search.best_estimator_
        xbg_mse[b] = np.mean(abs(target_only_xgb.predict(X_test) - Y_test)**2)

        ## Kernel Ridge Regression
        alpha_lst = (0.1 / X0.shape[0] * (3.0 ** np.array(range(-2,6))))
        param_grid = {'alpha': alpha_lst}
        target_only_krr = KernelRidge(kernel='rbf')
        grid_search = GridSearchCV(target_only_krr, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X0, Y0)
        target_only_krr = grid_search.best_estimator_
        krr_mse[b] = np.mean(abs(target_only_krr.predict(X_test) - Y_test)**2)

        ## Neural Network
        param_grid = {
            'hidden_layer_sizes': [(10,), (50,), (100,)],
            'alpha': [0.0001, 0.001, 0.01],
        }
        mlp = MLPRegressor(max_iter=200, random_state=0)
        grid_search = GridSearchCV(mlp, param_grid, cv=5)
        grid_search.fit(X0, Y0)
        target_only_mlp = grid_search.best_estimator_
        nn_mse[b] = np.mean(abs(target_only_mlp.predict(X_test) - Y_test)**2)

    target_only_mse = np.concatenate([xbg_mse, krr_mse, nn_mse], axis=0)
    res = pd.DataFrame(target_only_mse, columns=['MSE'])
    res['Method'] = np.repeat(['Target-only XGBoost', 'Target-only Kernel Ridge Regression', 'Target-only Neural Network'], B)
    res['Sample_size'] = [n_s] * 3 * B
    res_full = pd.concat([res_full, res], axis=0)
    res_full.to_csv('Results/Simulation_Concept_Covariate'+str(n_s)+'.csv', index=False)

## Oracle ML Regression

In [ ]:
B = 100
for n_s in [500, 1000, 2000, 5000]:
    res_full = pd.read_csv('Results/Simulation_Concept_Covariate'+str(n_s)+'.csv')
    xbg_mse = np.zeros(B)
    krr_mse = np.zeros(B)
    nn_mse = np.zeros(B)
    for b in range(B):
        d = 5
        np.random.seed(b)
        dat_source, dat0, dat0_full, dat_test = sim_data(n_s=n_s, n_0=50, n_test=5000, d=d, sig=0.5, mu_s=np.ones(d), mu_t=np.zeros(d), Sigma=np.eye(d), beta1=1/np.arange(1, d+1))

        # Oracle ML models
        X0_full = dat0_full[:, 1:]
        Y0_full = dat0_full[:, 0]
        X_test = dat_test[:, 1:]
        Y_test = dat_test[:, 0]

        ## XGBoost
        param_grid = {
            'learning_rate': [0.001, 0.01, 0.1],
            'n_estimators': [10, 50, 100], 
            'max_depth': [3, 5],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
        }
        xgb_model = XGBRegressor(objective='reg:squarederror', random_state=0)
        grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X0_full, Y0_full)
        target_only_xgb = grid_search.best_estimator_
        xbg_mse[b] = np.mean(abs(target_only_xgb.predict(X_test) - Y_test)**2)

        ## Kernel Ridge Regression
        alpha_lst = (0.1 / X0_full.shape[0] * (3.0 ** np.array(range(-2,6))))
        param_grid = {'alpha': alpha_lst}
        target_only_krr = KernelRidge(kernel='rbf')
        grid_search = GridSearchCV(target_only_krr, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X0_full, Y0_full)
        target_only_krr = grid_search.best_estimator_
        krr_mse[b] = np.mean(abs(target_only_krr.predict(X_test) - Y_test)**2)

        ## Neural Network
        param_grid = {
            'hidden_layer_sizes': [(10,), (50,), (100,)],
            'alpha': [0.0001, 0.001, 0.01],
        }
        mlp = MLPRegressor(max_iter=200, random_state=0)
        grid_search = GridSearchCV(mlp, param_grid, cv=5)
        grid_search.fit(X0_full, Y0_full)
        target_only_mlp = grid_search.best_estimator_
        nn_mse[b] = np.mean(abs(target_only_mlp.predict(X_test) - Y_test)**2)

    oracle_mse = np.concatenate([xbg_mse, krr_mse, nn_mse])
    res = pd.DataFrame(oracle_mse, columns=['MSE'])
    res['Method'] = np.repeat(['Oracle XGBoost', 'Oracle Kernel Ridge Regression', 'Oracle Neural Network'], B)
    res_full = pd.concat([res_full, res], axis=0)
    res_full.to_csv('Results/Simulation_Concept_Covariate'+str(n_s)+'.csv', index=False)

In [19]:
oracle_mse

array([[0.27121768, 0.26467678, 0.25556036],
       [0.26973405, 0.26496096, 0.25455988],
       [0.27139313, 0.27023601, 0.25679947]])

### ML Regression with Conditional Quantile Matching

In [6]:
B = 100
for n_s in [500, 1000, 2000, 5000]:
    res_full = pd.read_csv('Results/Simulation_Concept_Covariate'+str(n_s)+'.csv')
    xbg_mse = np.zeros(B)
    krr_mse = np.zeros(B)
    nn_mse = np.zeros(B)
    for b in range(B):
        d = 5
        np.random.seed(b)
        dat_source, dat0, dat0_full, dat_test = sim_data(n_s=1000, n_0=50, n_test=5000, d=d, sig=0.5, mu_s=np.ones(d), mu_t=np.zeros(d), Sigma=np.eye(d), beta1=1/np.arange(1, d+1))

        # ML models with transfer learning with conditional quantile matching
        X_dat0 = dat0[:, 1:]
        Y0 = dat0[:, 0]
        X_test = dat_test[:, 1:]
        Y_test = dat_test[:, 0]

        # Fit the engression generative model on each source data
        eng_mod = []
        X_source_tensor = []
        for i in range(len(dat_source)):
            Y_tensor = torch.tensor(dat_source[i][:, 0].reshape(-1,1), dtype=torch.float32)
            X_tensor = torch.tensor(dat_source[i][:, 1:], dtype=torch.float32)
            engressor = engression(X_tensor, Y_tensor, num_layer=2, hidden_dim=100, noise_dim=100, lr=0.0001, num_epochs=1000)
            X_source_tensor.append(X_tensor)
            eng_mod.append(engressor)
        X_dat0_tensor = torch.tensor(X_dat0, dtype=torch.float32)
        X_source_tensor = torch.cat(X_source_tensor, dim=0)

        # Sample response variables from each source data based on the covariates in the target domain
        N_sam = 3000
        Y0_sam = []
        for i in range(len(eng_mod)):
            Y0_sam.append(eng_mod[i].sample(X_dat0_tensor, sample_size=N_sam).detach().numpy().reshape(-1,1))
        Y0_sam = np.concatenate(Y0_sam, axis=1)
        Y0_sam_arr = np.concatenate([np.ones([Y0_sam.shape[0],1]), Y0_sam], axis=1)

        beta_sol = quantile_matching_estimate(np.repeat(Y0, N_sam), Y0_sam_arr, beta_init=None, stop_eps=1e-8, max_iter=1000, verbose=False)

        Y_source_pred = []
        for i in range(len(eng_mod)):
            Y_source_pred.append(eng_mod[i].predict(X_source_tensor, sample_size=100).detach().numpy().reshape(-1,1))
        Y_source_pred = np.concatenate(Y_source_pred, axis=1)
        Y_source_pred = np.concatenate([np.ones([Y_source_pred.shape[0],1]), Y_source_pred], axis=1)
        Y_matched = np.dot(Y_source_pred, beta_sol)

        # Kernel mean matching for covariate shift correction
        X_source = X_source_tensor.detach().numpy()
        kmm_weights = kernel_mean_matching(X_test, X_source, kern='rbf', B=10)[:,0]

        X_comb = np.concatenate([X_source, X_dat0], axis=0)
        Y_comb = np.concatenate([Y_matched, Y0], axis=0)
        weights = np.concatenate([kmm_weights, np.ones(X_dat0.shape[0])], axis=0)
        ## XGBoost
        param_grid = {
            'learning_rate': [0.001, 0.01, 0.1],
            'n_estimators': [10, 50, 100], 
            'max_depth': [3, 5],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
        }
        xgb_model = XGBRegressor(objective='reg:squarederror', random_state=0)
        grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X_comb, Y_comb, sample_weight=weights)
        target_only_xgb = grid_search.best_estimator_
        xbg_mse[b] = np.mean(abs(target_only_xgb.predict(X_test) - Y_test)**2)

        ## Kernel Ridge Regression
        alpha_lst = (0.1 / X_comb.shape[0] * (3.0 ** np.array(range(-2,6))))
        param_grid = {'alpha': alpha_lst}
        target_only_krr = KernelRidge(kernel='rbf')
        grid_search = GridSearchCV(target_only_krr, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X_comb, Y_comb, sample_weight=weights)
        target_only_krr = grid_search.best_estimator_
        krr_mse[b] = np.mean(abs(target_only_krr.predict(X_test) - Y_test)**2)

        ## Neural Network
        param_grid = {
            'hidden_layer_sizes': [(10,), (50,), (100,)],
            'alpha': [0.0001, 0.001, 0.01],
        }
        mlp = MLPRegressor(max_iter=200, random_state=0)
        grid_search = GridSearchCV(mlp, param_grid, cv=5)
        grid_search.fit(X_comb, Y_comb)
        target_only_mlp = grid_search.best_estimator_
        nn_mse[b] = np.mean(abs(target_only_mlp.predict(X_test) - Y_test)**2)

    tlcqm_mse = np.concatenate([xbg_mse, krr_mse, nn_mse])
    res = pd.DataFrame(oracle_mse, columns=['MSE'])
    res['Method'] = np.repeat(['TLCQM XGBoost', 'TLCQM Kernel Ridge Regression', 'TLCQM Neural Network'], B)
    res_full = pd.concat([res_full, res], axis=0)
    res_full.to_csv('Results/Simulation_Concept_Covariate'+str(n_s)+'.csv', index=False)

Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Batch is larger than half of the sample size. Training based on full-batch gradient descent.
[Epoch 1 (0%)] energy-loss: 0.6976,  E(|Y-Yhat|): 0.8664,  E(|Yhat-Yhat'|): 0.3376
[Epoch 100 (10%)] energy-loss: 0.6106,  E(|Y-Yhat|): 0.9160,  E(|Yhat-Yhat'|): 0.6107
[Epoch 200 (20%)] energy-loss: 0.5978,  E(|Y-Yhat|): 1.0119,  E(|Yhat-Yhat'|): 0.8283
[Epoch 300 (30%)] energy-loss: 0.5838,  E(|Y-Yhat|): 1.0759,  E(|Yhat-Yhat'|): 0.9841
[Epoch 400 (40%)] energy-loss: 0.5861,  E(|Y-Yhat|): 1.1561,  E(|Yhat-Yhat'|): 1.1401
[Epoch 500 (50%)] energy-loss: 0.5807,  E(|Y-Yhat|): 1.1506,  E(|Yhat-Yhat'|): 1.1398
[Epoch 600 (60%)] energy-loss: 0.5616,  E(|Y-Yhat|): 1.1547,  E(|Yhat-Yhat'|): 1.1862
[Epoch 700 (70%)] energy-loss: 0.5711,  E(|Y-Yhat|): 1.1489,  E(|Yhat-Yhat'|): 1.1556

[Epoch 100 (10%)] energy-loss: 0.6495,  E(|Y-Yhat|): 0.9408,  E(|Yhat-Yhat'|): 0.5826
[Epoch 200 (20%)] energy-loss: 0.6018,  E(|Y-Yhat|): 1.0338,  E(|Yhat-Yhat'|): 0.8641
[Epoch 300 (30%)] energy-loss: 0.6032,  E(|Y-Yhat|): 1.1164,  E(|Yhat-Yhat'|): 1.0264
[Epoch 400 (40%)] energy-loss: 0.5911,  E(|Y-Yhat|): 1.1802,  E(|Yhat-Yhat'|): 1.1781
[Epoch 500 (50%)] energy-loss: 0.5863,  E(|Y-Yhat|): 1.1854,  E(|Yhat-Yhat'|): 1.1982
[Epoch 600 (60%)] energy-loss: 0.5338,  E(|Y-Yhat|): 1.1715,  E(|Yhat-Yhat'|): 1.2755
[Epoch 700 (70%)] energy-loss: 0.5647,  E(|Y-Yhat|): 1.1958,  E(|Yhat-Yhat'|): 1.2622
[Epoch 800 (80%)] energy-loss: 0.5635,  E(|Y-Yhat|): 1.1795,  E(|Yhat-Yhat'|): 1.2320
[Epoch 900 (90%)] energy-loss: 0.5527,  E(|Y-Yhat|): 1.2039,  E(|Yhat-Yhat'|): 1.3023
[Epoch 1000 (100%)] energy-loss: 0.5851,  E(|Y-Yhat|): 1.2055,  E(|Yhat-Yhat'|): 1.2409

Training loss on the original (non-standardized) scale:
	Energy-loss: 0.8370,  E(|Y-Yhat|): 1.7603,  E(|Yhat-Yhat'|): 1.8466

Prediction-

In [21]:
tlcqm_mse

array([[0.31767674, 0.31296973, 0.31640316],
       [0.32529719, 0.32822047, 0.33123258],
       [0.31225785, 0.30412624, 0.31737129]])

## Pseudo-labeling Kernel Ridge Regression

In [39]:
# from mint.transfer_learning.KRR_pseudo_label import KRR_covariate_shift

In [38]:
# B = 3
# pskrr_mse = np.zeros(B)
# for b in range(B):
#     d = 5
#     np.random.seed(b)
#     dat_source, dat0, dat0_full, dat_test = sim_data(n_s=1000, n_0=50, n_test=5000, d=d, sig=0.5, mu_s=np.ones(d), mu_t=np.zeros(d), Sigma=np.eye(d), beta1=1/np.arange(1, d+1))

#     # ML models with pseudo-labeling with KRR
#     X_test = dat_test[:, 1:]
#     Y_test = dat_test[:, 0]

#     X_source = np.concatenate([dat_source[0][:, 1:], dat_source[1][:, 1:]], axis=0)
#     Y_source = np.concatenate([dat_source[0][:, 0], dat_source[1][:, 0]], axis=0)
#     print(X_source.shape)
#     krrPL = KRR_covariate_shift(n = X_source.shape[0], n_0 = X_test.shape[0], B = 1, sigma = 1, X = X_source, y = Y_source, 
#                                 X_0=X_test, y_0=np.ones([X_test.shape[0],]), seed=0)
#     krrPL.fit(rho = 0.1, beta = 2)
#     Y_pred = krrPL.predict_final(X_test)
#     pskrr_mse[b] = np.mean(abs(Y_pred - Y_test)**2)
#     print(pskrr_mse[b])

## Transfer Learning with Kernel Ridge Regression (TKRR)

In [31]:
# B = 3
# for n_s in [500, 1000, 2000, 5000]:
#     tkrr_mse = np.zeros(B)
#     for b in range(B):
#         d = 5
#         np.random.seed(b)
#         dat_source, dat0, dat0_full, dat_test = sim_data(n_s=1000, n_0=50, n_test=5000, d=d, sig=0.5, mu_s=np.ones(d), mu_t=np.zeros(d), Sigma=np.eye(d), beta1=1/np.arange(1, d+1))
#         # Prepare data
#         X_source = [dat[:, 1:] for dat in dat_source]
#         Y_source = [dat[:, 0] for dat in dat_source]
#         X0_train = dat0[:, 1:]
#         Y0_train = dat0[:, 0]
#         X0_test = dat_test[:, 1:]
#         Y0_test = dat_test[:, 0]

#         # Train KRR on combined data
#         X_comb = np.concatenate(X_source + [X0_train], axis=0)
#         Y_comb = np.concatenate(Y_source + [Y0_train], axis=0)
#         alpha_lst = 0.1 / X_comb.shape[0] * (3.0 ** np.arange(-3, 7))
#         param_grid = {'alpha': alpha_lst}
#         comb_krr = KernelRidge(kernel="rbf")
#         grid_search = GridSearchCV(comb_krr, param_grid, cv=5, scoring="neg_mean_squared_error")
#         grid_search.fit(X_comb, Y_comb)
#         comb_krr = grid_search.best_estimator_
#         # Fit residual model
#         Y0_pred = comb_krr.predict(X0_train)
#         Y_resi = Y0_train - Y0_pred
#         alpha_lst = 0.1 / X0_train.shape[0] * (3.0 ** np.arange(-3, 7))
#         param_grid = {"alpha": alpha_lst}
#         resi_krr = KernelRidge(kernel="rbf")
#         grid_search = GridSearchCV(resi_krr, param_grid, cv=5, scoring="neg_mean_squared_error")
#         grid_search.fit(X0_train, Y_resi)
#         resi_krr = grid_search.best_estimator_
#         # Predict on test data
#         Y0_pred_new = resi_krr.predict(X0_test) + comb_krr.predict(X0_test)
#         tkrr_mse[b] = np.mean(abs(Y0_pred_new - Y0_test) ** 2)

In [37]:
from sklearn.model_selection import train_test_split
from itertools import combinations
def fit_krr(X, Y, alpha_grid=None):
    if alpha_grid is None:
        alpha_grid = 0.1 / X.shape[0] * (3.0 ** np.arange(-3, 7))
    param_grid = {'alpha': alpha_grid}
    krr = KernelRidge(kernel="rbf")
    grid_search = GridSearchCV(krr, param_grid, cv=5, scoring="neg_mean_squared_error")
    grid_search.fit(X, Y)
    return grid_search.best_estimator_
def rkhs_norm(f1, f2, X):
    # Use L2 norm of predictions as a proxy for RKHS norm
    return np.linalg.norm(f1.predict(X) - f2.predict(X))


B = 3
for n_s in [500, 1000, 2000, 5000]:
    tkrr_mse = np.zeros(B)
    for b in range(B):
        d = 5
        np.random.seed(b)
        dat_source, dat0, dat0_full, dat_test = sim_data(n_s=1000, n_0=50, n_test=5000, d=d, sig=0.5, mu_s=np.ones(d), mu_t=np.zeros(d), Sigma=np.eye(d), beta1=1/np.arange(1, d+1))
        # Prepare data
        X_source = [dat[:, 1:] for dat in dat_source]
        Y_source = [dat[:, 0] for dat in dat_source]
        X0 = dat0[:, 1:]
        Y0 = dat0[:, 0]
        X0_test = dat_test[:, 1:]
        Y0_test = dat_test[:, 0]
        m = len(X_source)
        # --- Step 1: Split target data into T1 and T2 ---
        X0_T1, X0_T2, Y0_T1, Y0_T2 = train_test_split(X0, Y0, test_size=0.5, random_state=b)
        # Further split T2 into T21 and T22 for aggregation
        X0_T21, X0_T22, Y0_T21, Y0_T22 = train_test_split(X0_T2, Y0_T2, test_size=0.5, random_state=b)
        # --- Step 2: Fit KRR on each source and T1 ---
        fb0 = fit_krr(X0_T1, Y0_T1)  # Target model on T1
        fbk_list = [fit_krr(Xk, Yk) for Xk, Yk in zip(X_source, Y_source)]  # Source models
        # --- Step 3: Compute contrast functions and RKHS norms ---
        norms = [rkhs_norm(fbk, fb0, X0_T1) for fbk in fbk_list]
        ranks = np.argsort(norms)  # Indices of sources sorted by similarity
        # --- Step 4: Build candidate models using increasing numbers of sources ---
        candidate_models = [fb0]  # fb0 corresponds to Ab0 = ∅
        for ell in range(1, m+1):
            selected_indices = ranks[:ell]
            # Combine selected sources and T1
            X_comb = np.concatenate([X_source[i] for i in selected_indices] + [X0_T1], axis=0)
            Y_comb = np.concatenate([Y_source[i] for i in selected_indices] + [Y0_T1], axis=0)
            # Transferring step
            comb_krr = fit_krr(X_comb, Y_comb)
            # Debiasing step
            Y0_pred = comb_krr.predict(X0_T1)
            Y_resi = Y0_T1 - Y0_pred
            resi_krr = fit_krr(X0_T1, Y_resi)
            # Final model: sum of transferring and debiasing
            class CombinedModel:
                def __init__(self, m1, m2):
                    self.m1 = m1
                    self.m2 = m2
                def predict(self, X):
                    return self.m1.predict(X) + self.m2.predict(X)
            fb_ell = CombinedModel(comb_krr, resi_krr)
            candidate_models.append(fb_ell)
        # --- Step 5: Hyper-sparse aggregation (convex combination of at most two models) ---
        # Evaluate risk on T21 for each candidate
        risks = [np.mean((model.predict(X0_T21) - Y0_T21)**2) for model in candidate_models]
        best_idx = np.argmin(risks)
        # Find best convex combination of two models
        min_risk = risks[best_idx]
        best_combo = (best_idx, None, 1.0)
        for i, j in combinations(range(len(candidate_models)), 2):
            # Find optimal t in [0,1] for convex combination
            preds_i = candidate_models[i].predict(X0_T21)
            preds_j = candidate_models[j].predict(X0_T21)
            t_vals = np.linspace(0, 1, 101)
            for t in t_vals:
                preds = t * preds_i + (1-t) * preds_j
                risk = np.mean((preds - Y0_T21)**2)
                if risk < min_risk:
                    min_risk = risk
                    best_combo = (i, j, t)
        # Build final aggregated model
        i, j, t = best_combo
        class AggregatedModel:
            def __init__(self, m1, m2, t):
                self.m1 = m1
                self.m2 = m2
                self.t = t
            def predict(self, X):
                if self.m2 is None:
                    return self.m1.predict(X)
                return self.t * self.m1.predict(X) + (1-self.t) * self.m2.predict(X)
        if j is not None:
            fba = AggregatedModel(candidate_models[i], candidate_models[j], t)
        else:
            fba = AggregatedModel(candidate_models[i], None, t)
        # --- Step 6: Evaluate on test set ---
        Y0_pred_new = fba.predict(X0_test)
        tkrr_mse[b] = np.mean((Y0_pred_new - Y0_test) ** 2)
# print("SA-TKRR MSEs over B runs:", tkrr_mse)

SA-TKRR MSEs over B runs: [0.34925179 0.31861234 0.36970078]


In [35]:
tkrr_mse

array([0.34141852, 0.34105561, 0.32420118])

## Deep Transfer Learning for Conditional Shift in Regression (CDAR)

In [41]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Example TargetCNN definition for d=5
class TargetCNN(nn.Module):
    def __init__(self, d=5):
        super(TargetCNN, self).__init__()
        self.fc1 = nn.Linear(d, 384)
        self.fc2 = nn.Linear(384, 64)
        self.fc3 = nn.Linear(64, 16)
        self.predict = nn.Linear(16, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        inter_x1 = self.relu(x)
        x = self.fc2(inter_x1)
        inter_x2 = self.relu(x)
        x = self.fc3(inter_x2)
        inter_x3 = self.relu(x)
        result = self.predict(inter_x3)
        target_list = [inter_x1, inter_x2, inter_x3]
        return target_list, result

def to_tensor(X, Y):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    Y_tensor = torch.tensor(Y, dtype=torch.float32).reshape(-1, 1)
    return X_tensor, Y_tensor

In [42]:
import torch

def rbf_kernel(X, Y, gamma=0.4):
    # X: (n_samples_X, n_features)
    # Y: (n_samples_Y, n_features)
    X = X if X.ndim == 2 else X.view(X.size(0), -1)
    Y = Y if Y.ndim == 2 else Y.view(Y.size(0), -1)
    XX = torch.sum(X ** 2, 1).view(-1, 1)
    YY = torch.sum(Y ** 2, 1).view(1, -1)
    distances = XX + YY - 2 * torch.mm(X, Y.t())
    K = torch.exp(-gamma * distances)
    return K

def MLcon_kernel(source_list, source_pred, target_list, target_y, lamda=1.0):
    # Use only the first layer's features for simplicity
    X_p = source_list[0]  # (n_source, n_features)
    Y_p = source_pred     # (n_source, 1)
    X_q = target_list[0]  # (n_target, n_features)
    Y_q = target_y        # (n_target, 1)

    np_ = X_p.shape[0]
    nq_ = X_q.shape[0]
    I1 = torch.eye(np_, device=X_p.device)
    I2 = torch.eye(nq_, device=X_q.device)

    Kxpxp = rbf_kernel(X_p, X_p)
    Kxqxq = rbf_kernel(X_q, X_q)
    Kxqxp = rbf_kernel(X_q, X_p)
    Kypyq = rbf_kernel(Y_p, Y_q)
    Kyqyq = rbf_kernel(Y_q, Y_q)
    Kypyp = rbf_kernel(Y_p, Y_p)

    a = torch.mm(torch.inverse(Kxpxp + np_ * lamda * I1), Kypyp)
    b = torch.mm(a, torch.inverse(Kxpxp + np_ * lamda * I1))
    c = torch.mm(b, Kxpxp)
    out1 = torch.trace(c)

    a1 = torch.mm(torch.inverse(Kxqxq + nq_ * lamda * I2), Kyqyq)
    b1 = torch.mm(a1, torch.inverse(Kxqxq + nq_ * lamda * I2))
    c1 = torch.mm(b1, Kxqxq)
    out2 = torch.trace(c1)

    a2 = torch.mm(torch.inverse(Kxpxp + np_ * lamda * I1), Kypyq)
    b2 = torch.mm(a2, torch.inverse(Kxqxq + nq_ * lamda * I2))
    c2 = torch.mm(b2, Kxqxp)
    out3 = torch.trace(c2)

    out = out1 + out2 - 2 * out3
    return out

In [45]:
B = 3
cdar_mse = np.zeros(B)

for b in range(B):
    d = 5
    np.random.seed(b)
    dat_source, dat0, dat0_full, dat_test = sim_data(
        n_s=1000, n_0=50, n_test=5000, d=d, sig=0.5,
        mu_s=np.ones(d), mu_t=np.zeros(d), Sigma=np.eye(d), beta1=1/np.arange(1, d+1)
    )
    # Prepare data
    X_source = [dat[:, 1:] for dat in dat_source]
    Y_source = [dat[:, 0] for dat in dat_source]
    X0_train = dat0[:, 1:]
    Y0_train = dat0[:, 0]
    X0_test = dat_test[:, 1:]
    Y0_test = dat_test[:, 0]

    # Convert to tensors
    X_source_tensor, Y_source_tensor = to_tensor(X_source[0], Y_source[0])  # Use first source domain
    X0_train_tensor, Y0_train_tensor = to_tensor(X0_train, Y0_train)
    X0_test_tensor, Y0_test_tensor = to_tensor(X0_test, Y0_test)

    # Initialize model
    model = TargetCNN(d=d)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    criterion = nn.MSELoss()

    # CDAR hyperparameters
    Lambda = 1.0
    Beta = 1.0
    num_epochs = 200

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        # Forward pass for source and target
        source_list, source_pred = model(X_source_tensor)
        target_list, target_pred = model(X0_train_tensor)

        # Compute CEOD loss (replace with your actual implementation)
        CEOD_loss = MLcon_kernel(
        source_list, source_pred, target_list, Y0_train_tensor
        )

        # Hybrid loss
        loss = Lambda * criterion(target_pred, Y0_train_tensor) + Beta * CEOD_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Fold {b}, Epoch {epoch}, Loss: {loss.item()}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        _, y_pred = model(X0_test_tensor)
        y_pred = y_pred.numpy().flatten()
        mse = np.mean((Y0_test - y_pred) ** 2)
        cdar_mse[b] = mse
        print(f"Fold {b}, Test MSE: {mse:.4f}")

Fold 0, Epoch 0, Loss: 1.6087405681610107
Fold 0, Epoch 100, Loss: 0.16194851696491241
Fold 0, Test MSE: 0.4690
Fold 1, Epoch 0, Loss: 1.2023255825042725
Fold 1, Epoch 100, Loss: 0.16687558591365814
Fold 1, Test MSE: 0.4984
Fold 2, Epoch 0, Loss: 1.7064645290374756
Fold 2, Epoch 100, Loss: 0.20977719128131866
Fold 2, Test MSE: 0.6742


## Multi-Domain Adaptation for Regression under Conditional shift (DARC)

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class FeatureExtractor(nn.Module):
    def __init__(self, input_dim, domain_dim, feature_dim=2):
        super().__init__()
        self.domain_embed = nn.Embedding(domain_dim, 8)
        self.mlp = nn.Sequential(
            nn.Linear(input_dim + 8, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, feature_dim)
        )

    def forward(self, x, domain_id):
        domain_vec = self.domain_embed(domain_id)
        x_cat = torch.cat([x, domain_vec], dim=1)
        return self.mlp(x_cat)

def psp_loss(features, labels):
    dist_matrix = torch.cdist(features, features, p=2)
    label_matrix = torch.abs(labels.unsqueeze(0) - labels.unsqueeze(1))
    return nn.functional.mse_loss(dist_matrix, label_matrix)

class LinearRegressor(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.linear = nn.Linear(feature_dim, 1)

    def forward(self, features):
        return self.linear(features).squeeze(-1)

def train_feature_extractor(F, X, Y, domain_ids, epochs=20, lr=1e-3):
    F = F.to(device)
    optimizer = optim.Adam(F.parameters(), lr=lr)
    X = torch.tensor(X, dtype=torch.float32).to(device)
    Y = torch.tensor(Y, dtype=torch.float32).to(device)
    domain_ids = torch.tensor(domain_ids, dtype=torch.long).to(device)
    for epoch in range(epochs):
        F.train()
        features = F(X, domain_ids)
        loss = psp_loss(features, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (epoch+1) % 5 == 0:
            print(f"Epoch {epoch+1}, PSP Loss: {loss.item():.4f}")
    return F

def train_regressor(F, R, X, Y, domain_ids, epochs=10, lr=1e-3):
    F = F.to(device)
    R = R.to(device)
    optimizer = optim.Adam(R.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    X = torch.tensor(X, dtype=torch.float32).to(device)
    Y = torch.tensor(Y, dtype=torch.float32).to(device)
    domain_ids = torch.tensor(domain_ids, dtype=torch.long).to(device)
    for epoch in range(epochs):
        F.eval()
        with torch.no_grad():
            features = F(X, domain_ids)
        preds = R(features)
        loss = loss_fn(preds, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (epoch+1) % 5 == 0:
            print(f"Epoch {epoch+1}, Regressor Loss: {loss.item():.4f}")
    return R

In [50]:
B = 3
darc_mse = np.zeros(B)
d = 5

for b in range(B):
    np.random.seed(b)
    dat_source, dat0, dat0_full, dat_test = sim_data(
        n_s=1000, n_0=50, n_test=5000, d=d, sig=0.5,
        mu_s=np.ones(d), mu_t=np.zeros(d), Sigma=np.eye(d), beta1=1/np.arange(1, d+1)
    )
    # Multi-source domains
    num_source_domains = len(dat_source)
    # Stack all source domains
    X_source = np.vstack([dat[:, 1:] for dat in dat_source])
    Y_source = np.concatenate([dat[:, 0] for dat in dat_source])
    domain_ids_source = np.concatenate([
        np.full(len(dat_source[i]), i, dtype=int) for i in range(num_source_domains)
    ])
    # Target domain
    X_target = dat0[:, 1:]
    Y_target = dat0[:, 0]
    domain_ids_target = np.full(len(X_target), num_source_domains, dtype=int)  # Target domain ID
    # Combine for training
    X_train = np.vstack([X_source, X_target])
    Y_train = np.concatenate([Y_source, Y_target])
    domain_ids_train = np.concatenate([domain_ids_source, domain_ids_target])
    # Train DARC feature extractor
    F = FeatureExtractor(input_dim=d, domain_dim=num_source_domains+1, feature_dim=2)
    F = train_feature_extractor(F, X_train, Y_train, domain_ids_train, epochs=20, lr=1e-3)
    # Train linear regressor on constructed space
    R = LinearRegressor(feature_dim=2)
    R = train_regressor(F, R, X_train, Y_train, domain_ids_train, epochs=10, lr=1e-3)
    # Evaluate on test set (target domain)
    X_test = dat_test[:, 1:]
    Y_test = dat_test[:, 0]
    domain_ids_test = np.full(len(X_test), num_source_domains, dtype=int)
    X_test_torch = torch.tensor(X_test, dtype=torch.float32).to(device)
    domain_ids_test_torch = torch.tensor(domain_ids_test, dtype=torch.long).to(device)
    with torch.no_grad():
        features_test = F(X_test_torch, domain_ids_test_torch)
        preds = R(features_test).cpu().numpy()
    mse = np.mean((preds - Y_test)**2)
    darc_mse[b] = mse
    print(f"Fold {b+1}, Test MSE: {mse:.4f}")

Epoch 5, PSP Loss: 2.4587
Epoch 10, PSP Loss: 1.9449
Epoch 15, PSP Loss: 1.6350
Epoch 20, PSP Loss: 1.5797
Epoch 5, Regressor Loss: 2.7535
Epoch 10, Regressor Loss: 2.7392
Fold 1, Test MSE: 0.9207
Epoch 5, PSP Loss: 2.2825
Epoch 10, PSP Loss: 1.8244
Epoch 15, PSP Loss: 1.6174
Epoch 20, PSP Loss: 1.5314
Epoch 5, Regressor Loss: 5.6395
Epoch 10, Regressor Loss: 5.5880
Fold 2, Test MSE: 2.1952
Epoch 5, PSP Loss: 2.2956
Epoch 10, PSP Loss: 1.8233
Epoch 15, PSP Loss: 1.5867
Epoch 20, PSP Loss: 1.5437
Epoch 5, Regressor Loss: 2.4045
Epoch 10, Regressor Loss: 2.3868
Fold 3, Test MSE: 1.7316


In [51]:
darc_mse

array([0.92067408, 2.19517393, 1.73162177])